In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def linear_interpolation(x0, y0, x1, y1, x):
    """
    Perform linear interpolation to find the y value corresponding to x.

    Parameters:
    x0, y0: coordinates of the first point
    x1, y1: coordinates of the second point
    x: the x value to interpolate

    Returns:
    y: the interpolated y value
    """
    y = y0 + (y1 - y0) * (x - x0) / (x1 - x0)
    return y

# Example usage:
# x0, y0 = 0, 0
# x1, y1 = 10, 10
# x = 5
# y = linear_interpolation(x0, y0, x1, y1, x)
# print(f"The interpolated value at x={x} is y={y}")

## Given Constants

In [3]:

n_p = 300
n_g = None
n_g_guess = None

N_p = 15 # between 17 and 20
N_g = 45

D_p = None
D_g = None

P_d = 10

Power = 3 # Power in HP
Torque = None
life_time = 1000 # in hours?

phi = 20

F = 1.25
F_g = None
F_p = None

A_v = 8



## Calculations

In [4]:
#shock factor
k_o = 2.0

In [5]:
P_des = Power * k_o
P_des

6.0

### Velocity ratio, output speed

In [6]:
##Velocity ratio
if n_g_guess is not None:
    VR_guess = n_p/n_g_guess

VR_guess

NameError: name 'VR_guess' is not defined

In [1089]:
##Number of teeth on gear
if N_g is None:
    N_g = round(N_p * VR_guess)

N_g


45

In [1090]:
VR = N_g/N_p

if n_g is None:
    n_g = n_p / VR

VR, n_g

(3.0, 100.0)

Calculate cone angles

In [1091]:
#pinion cone angle

gamma = 180 / np.pi * np.arctan(N_p/N_g)

#gear cone angle

Gamma = 180 / np.pi * np.arctan(N_g/N_p)

assert(gamma + Gamma - 90 < 10**-8)

gamma, Gamma

(18.43494882292201, 71.56505117707799)

### Diameters, Center distances, pitch line speed

In [1092]:
##Diameters
if D_p is None:
    D_p = N_p / P_d
else:
    assert(False)

if D_g is None:
    D_g = N_g / P_d
else:
    assert(False)

D_p, D_g

(1.5, 4.5)

In [1093]:
##Center distance

C = (N_p + N_g) / (2 * P_d)
C

3.0

In [1094]:
##v_t

v_t = np.pi * D_p * n_p / 12
v_t

117.80972450961724

calc cone distances

In [1095]:
#outer cone distance

A_oG = D_g / (2 * np.sin(Gamma * np.pi / 180))

A_oP = D_p / (2 * np.sin(gamma * np.pi / 180))

A_oG, A_oP

(2.3717082451262845, 2.3717082451262845)

Calculate Face width

In [1096]:
#face width

if F is not None:
    F_p = F
    F_g = F

else:

    F_p = 0.3 * A_oP #nominal face width
    F_g = 0.3 * A_oG #nominal face width

    if F_p > 10 / P_d:
        F_p = 10 / P_d
        user_in = input("face width at max")

    if F_g > 10 / P_d:
        F_g = 10 / P_d
        user_in = input("face width at max")


F_p, F_g

(1.25, 1.25)

## bevel gear stuff follows

### Calculate forces

verify force firmulas in textbook

calc power, torque

In [1097]:

if Power is not None:
    Torque = 63000 * Power / n_p

if Torque is not None:
    Power = n_p * Torque / 63000

# assert(Power is not None); assert(Torque is not None)

Power, Torque

(3.0, 630.0)

calculate mean radius

In [1098]:
r_m = D_p / 2 - F_p / 2 * np.sin(gamma * np.pi / 180)

R_m = D_g / 2 - F_g / 2 * np.sin(Gamma * np.pi / 180)

r_m, R_m

(0.5523576462394763, 1.6570729387184289)

transmitted load

In [1099]:
W_t = Torque / r_m

wt2 = 33000 * Power / v_t

W_t, wt2
# assert(W_t - wt2 < 10**-8)

(1140.565364287293, 840.3380995252074)

In [1100]:
W_rp = W_t * np.tan(phi * np.pi / 180) * np.cos(gamma * np.pi / 180)
W_rp

393.8286457870289

In [1101]:
W_xp = W_t * np.tan(phi * np.pi / 180) * np.sin(gamma * np.pi / 180)
W_xp

131.276215262343

## choose a material - this part is sketch

seems to apply the same for all gears??

this part consists of finding numbers in tables

In [1102]:
# usually given - use table 9-5 in mott 
# A_v = 10
assert(A_v is not None)


In [1103]:
k_v = 1.08 #from A_v

assert(k_v > 0)

In [1104]:
#k_mb
cond = "0 straddle".upper()
assert(cond is not None)

k_ma = None

if cond == "2 straddle".upper():
    k_mb = 1
elif cond == "1 straddle".upper():
    k_mb = 1.1
elif cond == "0 straddle".upper():
    k_mb = 1.25

assert(k_mb is not None)

k_mb


1.25

In [1105]:
k_m = k_mb + 0.0036 * F**2
k_m

1.255625

### Find $k_s$ from a table idrk

In [1106]:
k_s = None
if P_d >= 16:
    k_s = 0.5

elif P_d < 16:
    k_s = 0.4867 + 0.2133 / P_d

assert(k_s is not None)
# assert(False) # idk if this is for shigley or mott
k_s

0.50803

### Find SF, reliability, $C_R, K_R, k_L, J$

In [1107]:
safety_factor = 1.0

In [1108]:
J_p = 0.235 # find from some random graph?
J_g = 0.235 # find from some random graph?

### Reliability > 99% implies the following bc of table 10-3

In [1109]:
#relaibility = 99% =>

k_R = 1.00
C_R = 1.0

## Calculate $Y_{np}, Y_{ng}$ with the above numbers

### Number of loading cycles

In [1110]:
#Number of cycles

N_cp = 60 * life_time * n_p
N_cg = 60 * life_time * n_g

print(f"N_cp: {N_cp:.2e}, N_cg: {N_cg:.2e}")

N_cp: 1.80e+07, N_cg: 6.00e+06


### Calcluate K_L

need a more accaurate measure for this

In [1111]:
k_Lp = None

if N_cp <= 5 * 10**6:
    k_Lp = 6.514 * N_cp**(-0.1192)
elif N_cp > 5 * 10**6:
    k_Lp = 1.3558 * N_cp**(-0.0178)


assert(k_Lp is not None)
k_Lp

1.007051727953924

In [1112]:
k_Lg = None

if N_cg <= 5 * 10**6:
    k_Lg = 6.514 * N_cg**(-0.1192)
elif N_cg > 5 * 10**6:
    k_Lg = 1.3558 * N_cg**(-0.0178)


assert(k_Lg is not None)
k_Lg

1.0269387398232055

## Calculate $S_{t}, S_{at}$

In [1113]:
k_o

2.0

In [1114]:
assert(F != 0)
assert(J != 0)

s_t = (W_t * P_d) / (F * J_p) * k_o * k_s * k_m * k_v
s_t

# W_t, P_d, F, J_p, k_o, k_s, k_m, k_B, k_v

53498.963497140016

In [1115]:
s_atp = s_t * safety_factor * k_R / k_Lp
s_atp

53124.345067990165

## Compute $C_{??} and $I$

In [1116]:
C_p = 2300 #for steel 

In [1117]:
#idk what this is
C_s = 0.125 * F + 0.4375
C_s

0.59375

In [1118]:
teeth = "crowned"

C_xc = None

if teeth == "crowned":
    C_xc = 1.5
elif teeth == "uncrowned":
    C_xc = 2
else:
    assert(False)

assert(C_xc is not None)

C_xc

1.5

In [1119]:
# from some random graph
I = 0.075

## Calculate $S_c$

In [1120]:
S_c = C_p * np.sqrt( (W_t * k_o * k_m * k_v * C_s * C_xc) / (F * D_p * I) ) #
S_c

321929.63066229323

## Find $C_L$

In [1121]:
C_Lp = None#1.0 # from spme random graph

if N_cp <= 10**4:
    C_Lp = 2.0
elif N_cp > 10**4:
    C_Lp = 3.4822 * N_cp**(-0.0602)

assert(C_Lp is not None)
C_Lp

1.273756186594274

In [1122]:
C_Lg = None#1.0 # from spme random graph

if N_cg <= 10**4:
    C_Lg = 2.0
elif N_cg > 10**4:
    C_Lg = 3.4822 * N_cg**(-0.0602)

assert(C_Lg is not None)
C_Lg

1.3608460777591285

## Find adjusted values of $S_c$

In [1123]:
S_acp = S_c * (safety_factor * C_R) / C_Lp
S_acp

252740.38630819746

## Follow remaining steps in spur gear guide

### Calculate the brinell hardness 

In [1136]:
# Prompt the user for input
HB_grade = input("HB grade 1 or 2? ")

Contact_HB = 0
Bending_HB = 0

if int(HB_grade) == 1:
    Contact_HB = ( S_acp /1000 - 23.62 ) / 0.341
    Bending_HB = ( s_atp / 1000 - 2.1 ) / 0.044 

elif int(HB_grade) == 2:
    Contact_HB = ( S_acp / 1000 - 29.56 ) / 0.3636
    Bending_HB = ( s_atp / 1000 - 5.980 ) / 0.048

else:
    assert(False)


print(f"Contact_HB: {Contact_HB:.4f} \nBending_HB: {Bending_HB:.4f} \n")


Contact_HB: 613.8074 
Bending_HB: 982.1739 



Use tables in the bible to find an appropriate material that satisfies the required HB. The material should be the same for the gear and the pinion.

Make sure to use table 9-8 form Mott to determine the heat treatment

Get $S_{acp}, S_{atp}$ for the material

In [1125]:
S_acp2 = None
S_atp2 = None

S_acg2 = None
S_atg2 = None

Need to find $Y_{**} $ and $ Z$

In [1126]:
contact_SF_p = S_acp2 * Y_np / (S_c * k_R)
bending_SF_p = S_atp2 * Y_np / (s_t * k_R)

print(f"Contact Safety Factor (Pinion): {contact_SF_p:f} \nBending Safety Factor (Pinion): {bending_SF_p:f}\n")


print(S_acp, Y_np, S_c, k_R)


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [243]:
# Power transmitting capacity

P_cap = n_p * F * I / (126000 * k_o * k_s * k_m * k_v) * ( (S_ac * D_p * Z_np) / (safety_factor * k_R * C_p) ) ** 2

print(f"P_cap = {P_cap:.4f} HP")

P_cap = 6.7464 HP
